# 🤗 트랜스포머를 이용한 텍스트 분석 모델 이용하기
---
파이썬(Python)을 이용한 자연어 처리(NLP, Natural Language Processing) 작업에서 최근 많이들 이용하는 라이브러리로 허깅페이스 트랜스포머(HuggingFace Transformers)를 빼놓을 수 없습니다. 이 라이브러리는 추상화 수준이 아주 높아서 허깅페이스에 올라와 있는 모델, 데이터셋, 토크나이저의 이름만 적으면 코드를 거의 변경하지 않고 비교 분석이 가능하다는 장점이 있지요. 특히, 딥러닝 프레임워크 중 점유율이 아주 높은 파이토치(PyTorch)와 텐서플로우(TensorFlow)를 가리지 않고 지원하기 때문에 더욱 확장성이 좋습니다.
</p></br></br>

이번에는 🤗 트랜스포머를 활용해서 다양한 텍스트 분석 모델을 이용하는 방법을 소개해 드리려고 합니다. 기본적인 사용법만 알아두면, 허깅페이스 홈페이지에서 내가 사용할 모델 이름만 확인해서 이 코드를 재활용할 수 있으니 자세히 읽어보세요!

* 이 글은 [허깅페이스 Transformers 둘러보기 문서](https://huggingface.co/docs/transformers/v4.45.1/ko/quicktour)를 참조하여 작성했습니다.
</p></br></br>

## 패키지 설치
---
만약 🤗 트랜스포머 설치를 하지 않았다면, 아래 명령어를 이용해 설치해 주시기 바랍니다. 파이토치와 텐서플로우도 함께 설치해 주시는 것이 좋습니다.
</p></br></br>

* 트랜스포머, 데이터셋 패키지 설치: `!pip install transformers datasets`
* 파이토치 설치: `!pip install torch`
* 텐서플로우 설치: `!pip install tensorflow`
</p></br></br>

## 영어 감정분석 모델
---
🤗 트랜스포머에는 파이프라인(pipeline) 이라고 하는 모듈이 있습니다. 이 모듈은 수행할 작업 및 모델 이름만 적으면 코드 몇 줄로 구현할 수 있게끔 만들어주는 강력한 기능을 보여준답니다. 이번에는 이 모듈을 이용해서 감정분석(sentiment analysis) 작업을 구현해 보겠습니다.
</p></br></br>
파이프라인은 기본적으로 두 개의 매개변수를 요구합니다. `task` 와 `model` 인데요, 각 매개변수는 아래와 같이 입력하시면 됩니다.

* task: [pipeline 모듈 문서](https://huggingface.co/docs/transformers/v4.45.1/en/main_classes/pipelines#transformers.pipeline.task) 에서 제공하는 task 정보에 따라 원하는 작업을 입력합니다.
* model: 허깅페이스에 업로드된 임의의 모델 이름을 입력해 줍니다. 이 때, 입력된 task를 지원하지 않는 모델은 이용할 수 없습니다.
</p></br></br>


In [1]:
from transformers import pipeline

# sentiment analysis의 기본 모델: distilbert/distilbert-base-uncased-finetuned-sst-2-english
classifier = pipeline("sentiment-analysis")

c:\Users\user\Desktop\Chunjae_edu\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\user\Desktop\Chunjae_edu\.conda\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space o

RuntimeError: Failed to import transformers.models.distilbert.modeling_tf_distilbert because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [2]:
# 단일 문장 입력
classifier(
    "Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do."
)

[{'label': 'NEGATIVE', 'score': 0.999713122844696}]

In [3]:
# 복수의 문장 입력
results = classifier(
    [
        "Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do.",
        "“Well!” thought Alice to herself, “after such a fall as this, I shall think nothing of tumbling down stairs!",
    ]
)

for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: NEGATIVE, with score: 0.9997
label: POSITIVE, with score: 0.7803


</p></br></br>

이 감정분석 모델은 입력된 문장을 POSITIVE or NEGATIVE의 두 가지 중 하나의 감정으로 분류한 뒤, label과 score 정보를 딕셔너리 형태로 반환합니다. 만약, 여러 문장을 한꺼번에 분석하고 싶다면 입력 데이터를 리스트로 묶어서 작성하시면 됩니다.
</p></br></br>

## 모델 및 토크나이저 선택하기
---
`pipeline()` 함수를 이용해 모델을 선택하는건 원하는 이름만 바꿔넣으면 되기 때문에 똑같은 코드를 이용해서 다양한 모델을 비교해볼 수 있다는 장점을 가집니다. 그런데, 토크나이저까지 임의로 선택하고 싶을 경우에는 `AutoTokenizer` 및 `AutoModelForSequenceClassification` 클래스를 이용할 수 있습니다. 여기서는 nlptown/bert-base-multilingual-uncased-sentiment 모델 및 토크나이저를 이용해 프랑스어를 처리할 수 있는 프로그램 만들기를 살펴보겠습니다.
</p></br></br>


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

C:\ProgramData\anaconda3\envs\test311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier(
    "Nous sommes très heureux de vous présenter la bibliothèque 🤗 Transformers."
)

[{'label': '5 stars', 'score': 0.7272652387619019}]